## 環境設定

参照: https://github.com/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/Item2vec.ipynb

In [ ]:
# データのDL，解凍，必要パッケージをインストール

# download and unzip dataset
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data/
!unzip -n data/ml-10m.zip -d ../data/
# install required packages
!pip install -r ../env_config/requirements.txt


In [2]:
import pandas as pd
import gensim
import logging
import os

# ファイルパス
DIR = os.path.dirname(os.path.abspath("__file__"))

# 学習ログ出力設定
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## データの前処理

In [3]:
# 授業ではユーザー数を1000に絞っていたが，本課題ではユーザー数を絞らずにモデルを学習することとする．

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(f'{DIR}/../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))

# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv(f'{DIR}/../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()

# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(f'{DIR}/../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')

# # データ量が多いため、ユーザー数を1000に絞って、試していく -> use all
# valid_user_ids = sorted(ratings.user_id.unique())[:1000]
# ratings = ratings[ratings["user_id"].isin(valid_user_ids)]

# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')
# print(movielens)
print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=69878, unique_movies=10677


In [4]:
# item2vecのインプットに使うデータの生成
item2vec_data = []
movielens_train_high_rating = movielens_train[movielens_train.rating >= 4]
for user_id, data in movielens_train_high_rating.groupby("user_id"):
    # 評価された順に並び替える
    # item2vecではwindowというパラメータがあり、itemの評価された順番も重要な要素となる
    item2vec_data.append(data.sort_values("timestamp")["movie_id"].tolist())

## モデルの学習

In [10]:
# 学習パラメータの設定

# 因子数
factors = 300
# エポック数
n_epochs = 60
# windowサイズ
window = 500
# スキップグラム
use_skip_gram = 1
# 階層的ソフトマックス
use_hierarchial_softmax = 0
# 使用する単語の出現回数のしきい値
min_count = 2

In [ ]:
# item2vecの学習

model = gensim.models.word2vec.Word2Vec(
    item2vec_data,
    vector_size=factors,
    window=window,
    sg=use_skip_gram,
    hs=use_hierarchial_softmax,
    epochs=n_epochs,
    min_count=min_count,
)

model.save(f"{DIR}/i2v_.model")

## movies.tsvのフィルタリング

In [23]:
# movies.tsvがもつ語彙を抽出
tsv_cols = ["movie_id","title","genre","tag"]
movies_tsv_vanilla = pd.read_csv(
    f'{DIR}/../data/movies.tsv',
    names = tsv_cols,
    sep = '\t',
    engine = 'python'
)
# ヘッダー削除
movies_tsv_vanilla = movies_tsv_vanilla.drop(0)
# データ型をstr -> intに変換
movies_tsv_vanilla['movie_id'] = movies_tsv_vanilla['movie_id'].astype(int)
vocab_tsv = set(movies_tsv_vanilla["movie_id"])

# モデルの語彙を抽出
model = gensim.models.word2vec.Word2Vec.load(f"{DIR}/../model/i2v.model")
vocab_model = set(model.wv.index_to_key)

# 差を取る．tsvの語彙集合の方が大きい．
delta = vocab_tsv - vocab_model
print(len(delta))

2023-09-10 00:17:02,130 : INFO : loading Word2Vec object from /home/tenk9/tmuY3S1/MachineLearning/mov_recommend/model/../model/i2v.model
2023-09-10 00:17:02,137 : INFO : loading wv recursively from /home/tenk9/tmuY3S1/MachineLearning/mov_recommend/model/../model/i2v.model.wv.* with mmap=None
2023-09-10 00:17:02,138 : INFO : setting ignored attribute cum_table to None
2023-09-10 00:17:02,191 : INFO : Word2Vec lifecycle event {'fname': '/home/tenk9/tmuY3S1/MachineLearning/mov_recommend/model/../model/i2v.model', 'datetime': '2023-09-10T00:17:02.191561', 'gensim': '4.3.2', 'python': '3.11.3 (main, Apr 19 2023, 23:54:32) [GCC 11.2.0]', 'platform': 'Linux-5.15.90.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'loaded'}


946


上記のように，movies.tsvにはitem2vecの語彙に存在しない映画がある．
このため，streamlitで語彙に存在しない映画を選択できてしまう．
これを回避するため，movies.tsvを修正する．

In [25]:
# modelの語彙に限定する
valid_movie_df = movies_tsv_vanilla[movies_tsv_vanilla['movie_id'].isin(vocab_model)]

# 整合性確認
print(vocab_model == set(valid_movie_df["movie_id"]))

True


In [26]:
# 保存
valid_movie_df.to_csv(f"{DIR}/../data/movies_valid.tsv", sep='\t', index=False)